In [28]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from io import BytesIO
from urllib.parse import urljoin, urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from PIL import Image
import os
import random
import re
import pandas as pd
import hashlib
import urllib.parse
from datetime import datetime
from json import dumps, loads
from shutil import copy2

In [2]:
def create_directories(base_url, categories, label='images'):
    # create the following dir struct; outputs > base website > categories
    base_dir = os.path.join('output', label, urlparse(base_url).netloc)
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
    
    for category, _ in categories:
        category_dir = os.path.join(base_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

    return base_dir

In [3]:
def get_articles_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    main_tag = soup.find('main', class_='main-content')
    info_headers = main_tag.find_all('header', class_='info-header')
    articles_links = []

    for header in info_headers:
        a_tag = header.find(class_='title').find('a')['href']
        link = a_tag if a_tag.startswith('https') else f'https://www.foxnews.com{a_tag}'
        articles_links.append(link)
    
    return articles_links

def article_scrapper(url):
    articles_links = get_articles_links(url)
    data = []

    def helper_scrapper(url):   
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'lxml')
        try:
            time = datetime.strptime(soup.find('time').text.split('EDT')[-2].strip(), "%B %d, %Y %I:%M%p")
            # time = soup.find('time').text.split('EDT')[-2].strip()
            headline = soup.find('h1').text
            content = soup.find('div', class_='article-body')
            images = [(img['alt'], img['src']) for img in content.find_all('img')]

            return headline, time, images
        except:
            return None
        
    for url in articles_links:   
        result = helper_scrapper(url)
        if result is not None:
            data.append(result)
    
    return data

In [4]:
def get_latest_articles(data, n=10):
    seen_headlines = set()
    unique_data = []

    for record in data:
        headline = record[0]
        if headline not in seen_headlines:
            seen_headlines.add(headline)
            unique_data.append(record)

    return sorted(unique_data, key=lambda x: x[1], reverse=True)[:n]

def download_image(img_url, save_dir, img_name):
    try:
        if not img_url.startswith('data:'):
            response = requests.get(img_url)
            img_data = response.content
            img = Image.open(BytesIO(img_data))
            width, height = img.size

            # Only save images larger than 100x100 pixels
            if width >= 100 and height >= 100:
                with open(os.path.join(save_dir, img_name), 'wb') as img_file:
                    img_file.write(img_data)
    except:
        pass

def download_images(category_url, save_dir, data):

    with open(os.path.join(save_dir, 'labels.csv'), 'w') as f:
        f.write('image number,alt,article_heading\n')
    
    records = []

    # parallising the downloads to make it faster
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        headlines = []
        for x, tuple in enumerate(data):
            headline, _, images_list = tuple
            for i, img in enumerate(images_list):
                alt_txt, img_url = img
                if alt_txt.startswith('Fox News'):
                    continue
                if img_url and not img_url.startswith('data:'):
                    img_url = urljoin(category_url, img_url)
                    combined_str = f"{alt_txt}{headline}".encode()
                    img_name = f'image_{x+1}_{i+1}.jpg'
                    records.append(f'{img_name},{alt_txt.replace(",", "")},{headline.replace(",", "")}\n')
                    futures.append(executor.submit(download_image, img_url, save_dir, img_name))
        
        with open(os.path.join(save_dir, 'labels.csv'), 'a') as f:
            f.writelines(records)
            
        for future in as_completed(futures):
            future.result()


In [5]:
categories = [
    ("Crime", "https://www.foxnews.com/category/us/crime"),
    ("Military", "https://www.foxnews.com/category/us/military"),
    ("Education", "https://www.foxnews.com/category/us/education"),
    ("Terror", "https://www.foxnews.com/category/us/terror"),
    ("Immigration", "https://www.foxnews.com/category/us/immigration"),
    ("Economy", "https://www.foxnews.com/category/us/economy"),
    ("Personal Freedoms", "https://www.foxnews.com/category/us/personal-freedoms"),
    ("Fox News Investigates", "https://www.foxnews.com/category/news-events/fox-news-investigates"),
    ("Digital Originals", "https://www.foxnews.com/category/us/digital-originals"),
    ("Executive", "https://www.foxnews.com/category/politics/executive"),
    ("Senate", "https://www.foxnews.com/category/politics/senate"),
    ("House", "https://www.foxnews.com/category/politics/house-of-representatives"),
    ("Judiciary", "https://www.foxnews.com/category/politics/judiciary"),
    ("Foreign Policy", "https://www.foxnews.com/category/politics/foreign-policy"),
    ("Polls", "https://www.foxnews.com/official-polls"),
    ("Elections", "https://www.foxnews.com/elections"),
    ("U.N.", "https://www.foxnews.com/category/world/united-nations"),
    ("Conflicts", "https://www.foxnews.com/category/world/conflicts"),
    ("Terrorism", "https://www.foxnews.com/category/world/terrorism"),
    ("Disasters", "https://www.foxnews.com/category/world/disasters"),
    ("Global Economy", "https://www.foxnews.com/category/world/global-economy"),
    ("Environment", "https://www.foxnews.com/category/world/environment"),
    ("Religion", "https://www.foxnews.com/category/world/religion"),
    ("Scandals", "https://www.foxnews.com/category/world/scandals"),
    ("FOX Nation Coverage", "https://www.foxnews.com/shows/fox-nation"),
    ("Entertainment", "https://www.foxnews.com/entertainment"),
    ("Celebrity News", "https://www.foxnews.com/category/entertainment/celebrity-news"),
    ("Movies", "https://www.foxnews.com/category/entertainment/movies"),
    ("TV News", "https://www.foxnews.com/category/entertainment/tv"),
    ("Music News", "https://www.foxnews.com/category/entertainment/music"),
    ("Style News", "https://www.foxnews.com/category/entertainment/style"),
    ("Entertainment Video", "https://www.foxnews.com/video/topics/entertainment"),
    ("Sports", "https://www.foxnews.com/sports"),
    ("NFL", "https://www.foxnews.com/category/sports/nfl"),
    ("College Football", "https://www.foxnews.com/category/sports/ncaa-fb"),
    ("MLB", "https://www.foxnews.com/category/sports/mlb"),
    ("NBA", "https://www.foxnews.com/category/sports/nba"),
    ("NHL", "https://www.foxnews.com/category/sports/nhl"),
    ("Golf", "https://www.foxnews.com/category/sports/golf"),
    ("Tennis", "https://www.foxnews.com/category/sports/tennis"),
    ("Soccer", "https://www.foxnews.com/category/sports/soccer"),
    ("UFC", "https://www.foxnews.com/category/sports/ufc"),
    ("WWE", "https://www.foxnews.com/category/organization/wwe"),
    ("Lifestyle", "https://www.foxnews.com/lifestyle"),
    ("Health", "https://www.foxnews.com/health"),
    ("Food & Drink", "https://www.foxnews.com/food-drink"),
    ("Auto", "https://www.foxnews.com/category/auto"),
    ("Travel & Outdoors", "https://www.foxnews.com/travel"),
    ("Real Estate", "https://www.foxnews.com/category/real-estate"),
    ("House & Home", "https://www.foxnews.com/category/house-and-home"),
    ("Style & Beauty", "https://www.foxnews.com/category/style-and-beauty"),
    ("Family", "https://www.foxnews.com/family"),
    ("Faith", "https://www.foxnews.com/category/faith-values/faith"),
    ("Science & Tech", "https://www.foxnews.com/tech"),
    ("Air & Space", "https://www.foxnews.com/category/science/air-and-space"),
    ("Security", "https://www.foxnews.com/category/tech/topics/security"),
    ("Innovation", "https://www.foxnews.com/category/tech/topics/innovation"),
    ("Planet Earth", "https://www.foxnews.com/category/science/planet-earth"),
    ("Wild Nature", "https://www.foxnews.com/category/science/wild-nature"),
    ("Video Games", "https://www.foxnews.com/category/tech/topics/video-games"),
    ("Military Tech", "https://www.foxnews.com/category/tech/topics/military-tech"),
    ("FOX News Shows", "https://www.foxnews.com/shows"),
    ("Programming Schedule", "https://www.foxnews.com/fnctv"),
    ("On Air Personalities", "https://www.foxnews.com/person/personalities"),
    ("Full Episodes", "https://www.foxnews.com/video/shows"),
    ("Show Clips", "https://www.foxnews.com/video/topics/v-most-recent-episodes"),
    ("News Clips", "https://www.foxnews.com/video/topics/latest-news-video"),
    ("Apps & Products", "https://www.foxnews.com/apps-products"),
    ("FOX News Go", "https://www.foxnews.com/go"),
    ("FOX Nation", "https://www.foxnews.com/shows/fox-nation"),
    ("Podcasts", "https://radio.foxnews.com/podcast/"),
]


base_url = 'https://www.foxnews.com/'

base_dir = create_directories(base_url, categories)

In [6]:
for category, category_url in tqdm(categories, desc='Downloading images for every category'):
    try:
        category_dir = os.path.join(base_dir, category)
        data = article_scrapper(f'{base_url}{category}')
        download_images(category_url, category_dir, get_latest_articles(data))
    except:
        continue

In [13]:
import shutil

for cat in os.listdir('output/images/www.foxnews.com'):
    if len(os.listdir(f'output/images/www.foxnews.com/{cat}')) == 1:
        shutil.rmtree(f'output/images/www.foxnews.com/{cat}')

In [64]:
df = pd.read_csv('labels.csv')
n, _ = df.shape

pairs = []

for i in range(n):
    for j in range(i + 1, n):
        pairs.append((list(df.iloc[i]), list(df.iloc[j])))

In [34]:
def create_file(base_dir):
    for category in os.listdir(base_dir):
        try:
            df = pd.read_csv(f'{base_dir}{category}/labels.csv')
            n, _ = df.shape

            pairs = []

            for i in range(n):
                for j in range(i + 1, n):
                    article_1 = int(re.search(r'\d+(?=_|$)',list(df.iloc[i])[0]).group())
                    article_2 = int(re.search(r'\d+(?=_|$)',list(df.iloc[j])[0]).group())
                    if article_1 != article_2:
                        pairs.append((list(df.iloc[i]), list(df.iloc[j])))

            with open(f'{base_dir.split("/")[2]}_pairs_{category}.csv', 'w') as f:
                for i, pair in enumerate(pairs):
                    p1, p2 = pair
                    img1, alt1, headline1 = p1
                    img2, alt2, headline2 = p2
                    f.write(f'{i+1},{headline1},{headline2}\n')
                    f.write(f',https://raw.githubusercontent.com/ayainfida/news-scrapper/main/output/images/www.foxnews.com/{urllib.parse.quote(category)}/{img1},https://raw.githubusercontent.com/ayainfida/news-scrapper/main/output/images/www.foxnews.com/{urllib.parse.quote(category)}/{img2}\n')
                    f.write(f',{alt1},{alt2}\n')

        except:
            continue

In [35]:
import re

create_file('output/images/www.foxnews.com/')

In [39]:
num_pairs = []

for file in os.listdir():
    if file.startswith('www.foxnews.com_pairs'):
        try:
            df = pd.read_csv(file)
            num_pairs.append((int(list(df.iloc[-3])[0]), file))
        except:
            continue

In [40]:
sorted_array = sorted(num_pairs, key=lambda x: x[0])
sorted_array

[(460, 'www.foxnews.com_pairs_Golf.csv'),
 (607, 'www.foxnews.com_pairs_Immigration.csv'),
 (621, 'www.foxnews.com_pairs_Education.csv'),
 (637, 'www.foxnews.com_pairs_Executive.csv'),
 (663, 'www.foxnews.com_pairs_Economy.csv'),
 (664, 'www.foxnews.com_pairs_Disasters.csv'),
 (722, 'www.foxnews.com_pairs_Environment.csv'),
 (785, 'www.foxnews.com_pairs_Crime.csv'),
 (906, 'www.foxnews.com_pairs_House.csv'),
 (990, 'www.foxnews.com_pairs_Lifestyle.csv'),
 (1121, 'www.foxnews.com_pairs_Faith.csv')]

In [106]:
import shutil
cat = []

for file in os.listdir():
    if file.startswith('www.foxnews.com') and file.endswith('.csv'):
        cat.append(file.replace('.csv', '').split('_')[-1])

for dir in os.listdir('output/images/www.foxnews.com'):
    if dir not in cat:
        shutil.rmtree(f'output/images/www.foxnews.com/{dir}')